In [ ]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, datasets 
import time
import random
from AddFunc import

AttributeError: module 'torch' has no attribute 'version'

This code classifies 10 different classes of images

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convL1 = nn.Conv2d(in_channels=3,out_channels=64, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL1.weight)
        self.convL2 = nn.Conv2d(in_channels=64,out_channels=48, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL2.weight)
        self.convL3 = nn.Conv2d(in_channels=48,out_channels=32, 
                                kernel_size=(2,2), stride= (1,1),padding=1)
        nn.init.kaiming_normal(self.convL1.weight)


        self.linL4 = nn.Linear(1568, 10, bias=True)
        nn.init.xavier_uniform_(self.linL4.weight)
        self.pool3 = nn.MaxPool2d((3,3), (3,3))
        self.pool2 = nn.MaxPool2d((2,2), (2,2))


        self.relu = nn.ReLU()  
        self.Batch_norm = nn.BatchNorm1d(num_features=32)
        self.Lrelu = nn.LeakyReLU()
        self.softplus = nn.Softplus()
        self.Softmax = nn.Softmax()

    def forward(self, x):
        self.Batch_norm(x)
        x=self.Lrelu(self.convL1(x))
        x=self.pool3(x)
        x=self.relu(self.convL2(x))
        x=self.pool2(x)
        x=self.relu(self.convL3(x))

        x=torch.flatten(x)
        x=self.Softmax(self.linL4(x))
        
        return x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Additional Functions

In [12]:
def ret_shape(training_set)->str:
    ret_string = ""
    for idx, data in enumerate(training_set):
            datas = data[0]
            labels = data[1]
            ret_string+="{}\n".format(datas.shape)
            ret_string+="Labels:{}\n".format(labels)
            ret_string+="Labels shape:{}\n".format(len(labels))
            ret_string+="Labels[0] shape:{}\n".format(labels[0].shape)
            break
    return ret_string


def max_index(max_tnsr):
    max_tnsr = max_tnsr.tolist()
    max_num = 0
    for num in max_tnsr:
        if max_num < num:
            max_num = num
    return max_tnsr.index(max_num)

def val_func(convNet, cnn_num,valDS, dev): # Selects the best network
                                                #   out of n networks
    cnns_loss = [0]*cnn_num # represents the 10 cnns
    print("\n| Starting validation set run |\n")
    for val_sample in valDS:
        for j in range(cnn_num):
            prediction = convNet[j].forward(torch.squeeze(val_sample[0].to(dev)))
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(prediction, val_sample[1].to(dev))
            cnns_loss[j]+=loss # Adds the loss of the
    cnns_loss = [j*-1 for j in cnns_loss]
    return convNet[max_index(cnns_loss)]# picks the cnn with the lowest loss

def train_one_epoch(cnn, FILE_PATH, train_ds, optimizer, dev, epoch_iter):
    iter = 0


    # Training loop part
    for sample in train_ds:
        cnn = cnn.to(dev) # Added due to errors (Should remain in the loop)
        targetY = torch.nn.functional.one_hot(
            torch.tensor(sample[1]), num_classes=10)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
        targetY = torch.squeeze(targetY).to(dev)
        
        pred = cnn.forward(torch.squeeze(sample[0]).to(dev))
        loss_func = nn.CrossEntropyLoss()
        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(cnn.cpu(), FILE_PATH)
            grad_tmp = cnn.linL4.weight.grad
            print(iter,"({})".format(epoch_iter+1), " | ",grad_tmp.sum()**2/len(grad_tmp))

        optimizer.zero_grad()
        iter+=1

Train Cell:

In [13]:
FILE_PATH = "model.pth"

def train_net(cnns, train_ds, val_ds, dev):
    iter = 0
    epoch_num = 4 # Determines the epoch number
    

    # Pre-train and validation phase
    valtr_num = 2000 # The scale of the minibatch used to train the CNNs that are to be validated
    val_train, reg_train = random_split(train_ds, [valtr_num, len(train_ds)-valtr_num])
    for uno_cnn in cnns:
        optimizer = optim.SGD(params=uno_cnn.parameters(), lr=6.9e-4,
                          momentum=0.9,
                          weight_decay=1e-4)
        train_one_epoch(uno_cnn, FILE_PATH, val_train, optimizer, dev, -1)

    # Multiple epoch training phase
    for i in range(epoch_num):
        optimizer = optim.SGD(params=cnn.parameters(), lr=6.9e-4,
                          momentum=0.9,
                          weight_decay=1e-4)
        train_one_epoch(cnn, FILE_PATH, train_ds, optimizer, dev,i)


In [ ]:
if __name__ == '__main__':

    # Accessing processing unit
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)
    net = CNN().to(device)

    # Loading the datasets
    train_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=True, transform=transforms.Compose([transforms.ToTensor()])
        #, target_transform=transforms.Compose([transforms.ToTensor()])
    )
    train_data = DataLoader(train_data, shuffle=True)

    test_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=False, transform=transforms.Compose([transforms.ToTensor()]), 
    )
    val_data, test_data = random_split(test_data, [1000, len(test_data)-1000])

    
    train_net(net, train_data, val_data,dev=device)


Test Cell:

In [ ]:
FILE = "model.pth"
model = torch.load(FILE)
model.eval()



image_lst = [0]*10
right_pred = 0
for i in range(len(test_data)):
    if max_index(model.forward(test_data[i][0]))==test_data[i][1]:
        right_pred+=1
    image_lst[max_index(model.forward(test_data[i][0]))]+=1
print("model's accuracy:",right_pred/100,"%")
print(image_lst)